## 데이터로드

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

# plotting 환경 설정
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
plt.rcParams['figure.figsize'] = (5,5) # 그래프 크기
plt.rcParams['font.family'] = 'Malgun Gothic' # 글꼴
plt.rcParams['font.size'] = 12 # 글꼴 크기
plt.rcParams['axes.unicode_minus'] = False # 마이너스 표기

# 그래프가 노트북 안에 보이게 하기 위해
%matplotlib inline

In [ ]:
vod = pd.read_csv('../data/전처리data/vod_prepro_series_nm.csv')

In [2]:
# 데이터를 구분하는 문자를 지정하는 옵션
# 제목만 전처리한 항목
vod = pd.read_csv('../data/전처리data/vod_prepro_series_nm.csv')
con = pd.read_csv('../data/전처리data/con_prepro_series_nm.csv', encoding='cp949')
vod_id = pd.read_csv('../data/전처리data/vod_id_origin.csv', encoding = 'cp949')

In [3]:
vod_id.head(1)

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,rename,counts,vod_id
0,와이낫크루 뜻밖의 여행,NaN,"KCM,권은비,조정식,박현규",NaN,TV 연예/오락,기타,TV프로그램,69분,"# 오늘, 이 동네는 우리가 알잘딱깔센 접수한다! 뻔한 랜드마크는 궁금하지도 않고,...",와이낫크루뜻밖의여행,16,0


# click

In [65]:
content = con.copy()
content.head(3)

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt,Category,rename,vod_id
0,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1,20230813071155,키즈,핑크퐁동물동요,882
1,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1,20230813071215,키즈,핑크퐁동물동요,882
2,63086000,지니강이 플러스 시즌7,지니강이 플러스 시즌7,키즈,기타,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,"헤이지니,럭키강이",11,20230827145542,키즈,지니강이플러스시즌7,1622


In [66]:
# clicked = subsr + series_nm 클릭횟수
# count = 전체 클릭 횟수
# click = click/count
clicked = content.groupby(['subsr', 'vod_id']).size().reset_index(name='clicked').sort_values(by='clicked', ascending=False)
clicked

,subsr,vod_id,clicked
3470,64154000,754,232
399,60326000,908,119
4309,64720000,336,110
4281,64680000,1166,105
3678,64385000,16,104
...,...,...,...
4406,64776000,3803,1
4408,64776000,4109,1
4409,64776000,4673,1
4410,64776000,4678,1


In [67]:
count = con.groupby(['subsr']).size().reset_index(name='count').sort_values(by='count', ascending=False)
count

,subsr,count
347,64154000,853
376,64385000,837
97,60939000,759
50,60326000,759
67,60593000,632
...,...,...
451,64937000,1
670,66182000,1
123,61223000,1
663,66144000,1


In [68]:
n_con = pd.merge(clicked, count, on='subsr', how='left')
n_con

,subsr,vod_id,clicked,count
0,64154000,754,232,853
1,60326000,908,119,759
2,64720000,336,110,205
3,64680000,1166,105,207
4,64385000,16,104,837
...,...,...,...,...
7338,64776000,3803,1,53
7339,64776000,4109,1,53
7340,64776000,4673,1,53
7341,64776000,4678,1,53


In [69]:
n_con[n_con['count'].isna()]

,subsr,vod_id,clicked,count


In [70]:
n_con["c_score"] = n_con["clicked"]/n_con["count"]
score = n_con[['subsr', 'vod_id', 'c_score']]
score

,subsr,vod_id,c_score
0,64154000,754,0.271981
1,60326000,908,0.156785
2,64720000,336,0.536585
3,64680000,1166,0.507246
4,64385000,16,0.124253
...,...,...,...
7338,64776000,3803,0.018868
7339,64776000,4109,0.018868
7340,64776000,4673,0.018868
7341,64776000,4678,0.018868


In [71]:
score.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7343 entries, 0 to 7342
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   subsr    7343 non-null   int64  
 1   vod_id   7343 non-null   int64  
 2   c_score  7343 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 229.5 KB


In [72]:
score[score['c_score'].isna()]

,subsr,vod_id,c_score


In [73]:
content.head(1)

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt,Category,rename,vod_id
0,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1,20230813071155,키즈,핑크퐁동물동요,882


In [107]:
clicking = pd.merge(content, score, on=['subsr', 'vod_id'], how='left')
click = clicking[['subsr', 'series_nm', 'vod_id', 'ct_cl', 'genre_of_ct_cl', 'Category', 'c_score', 'rename']].drop_duplicates()
click

,subsr,series_nm,vod_id,ct_cl,genre_of_ct_cl,Category,c_score,rename
0,65968000,핑크퐁 동물 동요,882,키즈,기타,키즈,1.000000,핑크퐁동물동요
2,63086000,지니강이 플러스 시즌7,1622,키즈,기타,키즈,0.012195,지니강이플러스시즌7
3,63086000,지니강이 플러스 시즌9,2043,키즈,기타,키즈,0.006098,지니강이플러스시즌9
4,62733000,엘리가간다 시즌12,1958,키즈,기타,키즈,0.500000,엘리가간다시즌12
5,63086000,지니강이 플러스 시즌8,1596,키즈,기타,키즈,0.012195,지니강이플러스시즌8
...,...,...,...,...,...,...,...,...
21110,64982000,포켓몬스터 XY and Z,2918,키즈,애니메이션,키즈,0.014493,포켓몬스터XYandZ
21111,66864000,포켓몬스터 썬 앤 문 3,3135,키즈,애니메이션,키즈,0.045455,포켓몬스터썬앤문3
21120,59917000,라바 인 뉴욕,10,키즈,오락,키즈,0.500000,라바인뉴욕
21121,63086000,사이틴 시즌1,3468,키즈,학습,키즈,0.006098,사이틴시즌1


In [108]:
click[click['c_score'].isna()]

,subsr,series_nm,vod_id,ct_cl,genre_of_ct_cl,Category,c_score,rename


In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(click['c_score'], bins=30, edgecolor='black')  # Adjust the number of bins as needed
plt.show()

In [109]:
click.to_csv('../data/전처리data/click.csv', index=False, encoding='cp949')

# click_plus

In [110]:
filtered = pd.read_csv('../data/전처리data/filtered_rows.csv')
filtered.head(1)

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,66424000,(HD)나 혼자산다 505회(23/07/28),TV 연예/오락,기타,1452,네가 알던 내가 아냐 편. 무지개 회원들의 반전 가득한 하루! 쇼 음악중심 스페셜 ...,"전현무,기안84,박나래,키,코드쿤스트",1:32,20230831182001


In [111]:
data = filtered.copy()
# 무삭제 제거
data['asset_nm'] = data['asset_nm'].str.replace(r'무삭제판|무삭제', '', regex = True)

# 예고편 제거
data = data[~data['asset_nm'].str.contains(r'\(예고편\)|\(예고\)', regex=True)]

# 예약구매, 사전구매 제거
data = data[~data['asset_nm'].str.contains('예약구매|사전구매')]

# 00회가 포함된 행은 런타임이 0또는 1이므로 제거
data = data[~data['asset_nm'].str.contains(r'\b00회\b')]

# 괄호와 그 안의 내용 제거
data['asset_nm'] = data['asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
data['asset_nm'] = data['asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
data['asset_nm'] = data['asset_nm'].str.replace(r'\.\.\.', '', regex=True)
data['asset_nm'] = data['asset_nm'].str.replace(r'\…', '', regex=True) # … 기호 제거
data['asset_nm'] = data['asset_nm'].str.rstrip('.')
data['asset_nm'] = data['asset_nm'].str.replace('-', " ")
data['asset_nm'] = data['asset_nm'].str.strip()

data['asset_nm'] = data['asset_nm'].str.replace(r'\d+회$', '', regex=True)
data['asset_nm'] = data['asset_nm'].str.replace(r'\d+회\.', '', regex=True)
data['asset_nm'] = data['asset_nm'].str.replace(r'\d+화$', '', regex=True)
data['asset_nm'] = data['asset_nm'].str.strip()
data['asset_nm'] = data['asset_nm'].str.rstrip('.')

# 셋탑번호가 66056000인 것 삭제 - content 에서 이상치 아이디로 판별
data = data[data['subsr'] != 66056000]

data['rename'] = data['asset_nm'].copy()

data['category'] = data['ct_cl'].apply(lambda x : x if x in ['영화', '키즈'] else 'TV프로그램')
data['rename'] = data['rename'].apply(lambda x : x.replace(' ', '') if isinstance(x, str) else x)

data.tail(1)

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt,rename,category
790,66317000,알사탕,키즈,학습,200,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,0:08,20230903223210,알사탕,키즈


In [112]:
data = data[['subsr', 'asset_nm', 'ct_cl', 'genre_of_ct_cl', 'rename', 'category']]
data

,subsr,asset_nm,ct_cl,genre_of_ct_cl,rename,category
0,66424000,나 혼자산다,TV 연예/오락,기타,나혼자산다,TV프로그램
1,66620000,런닝맨,TV 연예/오락,기타,런닝맨,TV프로그램
2,66620000,런닝맨,TV 연예/오락,기타,런닝맨,TV프로그램
3,63665000,내 딸 금사월,TV드라마,기타,내딸금사월,TV프로그램
4,66875000,내 눈에 콩깍지,TV드라마,기타,내눈에콩깍지,TV프로그램
...,...,...,...,...,...,...
786,66317000,호랑이 뱃속 잔치,키즈,학습,호랑이뱃속잔치,키즈
787,66317000,호랑이 뱃속 잔치,키즈,학습,호랑이뱃속잔치,키즈
788,66317000,이파라파냐무냐무,키즈,학습,이파라파냐무냐무,키즈
789,66317000,알사탕,키즈,학습,알사탕,키즈


In [113]:
# id 병합
id = vod_id[['rename', 'Category', 'vod_id']]
id.columns = ['rename', 'category', 'vod_id']
id

,rename,category,vod_id
0,와이낫크루뜻밖의여행,TV프로그램,0
1,그레이트뷰티,영화,1
2,해안선,영화,2
3,아는형님,TV프로그램,3
4,베놈,영화,4
...,...,...,...
4710,베놈2:렛데어비카니지,영화,4710
4711,성월동화,영화,4711
4712,다큐인사이트코드블루,TV프로그램,4712
4713,캠프제로베이스원,TV프로그램,4713


In [114]:
df = data.merge(id, on = ['rename', 'category'], how = 'left')
plus = df[['subsr', 'asset_nm', 'vod_id', 'ct_cl', 'genre_of_ct_cl', 'category', 'rename']]
plus.columns = ['subsr', 'series_nm', 'vod_id', 'ct_cl', 'genre_of_ct_cl', 'category', 'rename']
plus = plus[['subsr', 'series_nm', 'vod_id', 'ct_cl', 'genre_of_ct_cl', 'category', 'rename']].drop_duplicates()
plus['c_score'] = 1
plus

,subsr,series_nm,vod_id,ct_cl,genre_of_ct_cl,category,rename,c_score
0,66424000,나 혼자산다,432,TV 연예/오락,기타,TV프로그램,나혼자산다,1
1,66620000,런닝맨,296,TV 연예/오락,기타,TV프로그램,런닝맨,1
3,63665000,내 딸 금사월,1157,TV드라마,기타,TV프로그램,내딸금사월,1
4,66875000,내 눈에 콩깍지,490,TV드라마,기타,TV프로그램,내눈에콩깍지,1
6,66875000,금이야 옥이야,784,TV드라마,기타,TV프로그램,금이야옥이야,1
...,...,...,...,...,...,...,...,...
774,60463000,엄지 아가씨,694,키즈,기타,키즈,엄지아가씨,1
775,65724000,라바 시즌1,55,키즈,오락,키즈,라바시즌1,1
778,65724000,라바 인 뉴욕,10,키즈,오락,키즈,라바인뉴욕,1
783,66317000,꽁꽁꽁,1036,키즈,학습,키즈,꽁꽁꽁,1


In [98]:
plus.to_csv('../data/전처리data/content_plus_rename.csv', encoding='cp949', index=False)

In [115]:
# 'c_score'가 있는 경우에는 concat, 없는 경우에는 1로 채우기
click.rename(columns={'Category': 'category'}, inplace=True)
click_plus = pd.concat([click, plus], axis=0, join='outer').fillna({'c_score': 1}).reset_index(drop=True)
click_plus = click_plus.sort_values(by = 'subsr').reset_index(drop = True)
click_plus

,subsr,series_nm,vod_id,ct_cl,genre_of_ct_cl,category,c_score,rename
0,59879000,낭만닥터 김사부2,1168,TV드라마,기타,TV프로그램,0.111111,낭만닥터김사부2
1,59879000,소방서 옆 경찰서,2092,TV드라마,기타,TV프로그램,0.333333,소방서옆경찰서
2,59879000,신성한 이혼,1190,TV드라마,기타,TV프로그램,0.111111,신성한이혼
3,59879000,나의 해방일지,1859,TV드라마,기타,TV프로그램,0.111111,나의해방일지
4,59879000,나쁜엄마,3403,TV드라마,기타,TV프로그램,0.111111,나쁜엄마
...,...,...,...,...,...,...,...,...
7553,67160000,스파이더맨: 어크로스 더 유니버스,2147,영화,애니메이션,영화,0.125000,스파이더맨:어크로스더유니버스
7554,67160000,비공식작전,2057,영화,드라마,영화,0.125000,비공식작전
7555,67161000,스파이 코드명 포춘,3824,영화,액션/어드벤쳐,영화,1.000000,스파이코드명포춘
7556,67164000,미션 임파서블: 데드 레코닝 PART ONE,2867,영화,액션/어드벤쳐,영화,0.250000,미션임파서블:데드레코닝PARTONE


In [116]:
click_plus.to_csv('../data/전처리data/click_plus.csv', encoding='cp949', index=False)